In [26]:
from datasets import load_metric
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")

import pandas as pd
import os
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor, Wav2Vec2CTCTokenizer
import soundfile as sf
import os
# Use old or new vocab? 
os.system("cp vocab_300_with_numbers.json vocab.json")



tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(
        "/home/peterr/macocu/task8/", unk_token="[UNK]", 
        pad_token="[PAD]", word_delimiter_token=" ")
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor, tokenizer=tokenizer)
model = Wav2Vec2ForCTC.from_pretrained("/home/peterr/macocu/task8/31_normalised_220k/checkpoint-6500").cuda()

def transcribe(path):
    speech, sample_rate = sf.read(path)
    input_values = processor(speech, sampling_rate=sample_rate, return_tensors="pt").input_values.cuda()

    logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0]).lower()
    return transcription


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [27]:
import pandas as pd

df = pd.read_csv("35_eval_split.csv")
df.columns

Index(['hashname', 'orig_file', 'start', 'end', 'words', 'word_start_times',
       'norm_words', 'norm_words_start_times', 'utterance_id_start',
       'utterance_id_end', 'speaker_info', 'split', 'norm_words_edited',
       'sentence', 'Speaker_role', 'Speaker_type', 'Speaker_party',
       'Speaker_party_name', 'Party_status', 'Speaker_name', 'Speaker_gender',
       'Speaker_birth'],
      dtype='object')

In [28]:
import os
df["path"] = df.hashname.apply(lambda s: os.path.join("/home/peterr/macocu/task8/transfer", s))
df["model_transcript"] = df.path.apply(transcribe)

In [29]:
df.to_csv("36_eval_split_with_full_data_and_transcription.csv", index=False)

In [30]:

pd.set_option('display.max_colwidth', None)
df[["path","sentence", "model_transcript"]]

,path,sentence,model_transcript
0,/home/peterr/macocu/task8/transfer/Ki_SnDM_EkQ_2917.58-2937.5.wav,izraelska tvrtka prodaje avione hrvatskoj direktni poslovni partner king icta s kojega je došao ministar obrane i direktni poslovni partner airo partner koji je dakle optužen i je sudjelovao u dodijeli novca u kešu dakle mito,iizrelatska tvrtka prodajavione hrvatskoj djirektni posoni parnevi igintcas koji je dakle skoji go je došio ministarobrne i direktni posovani partnje raro partnera koji je dakle optužen i je sudjelovo u dodjeli novca ukešo dakle mitu
1,/home/peterr/macocu/task8/transfer/XguZsDKdRh4_13797.59-13811.74.wav,jučer trećem sutra ili prekosutra četvrtoj osobi dakle ne samo što samo kao saborski zastupnici uživamo u činjenici da posjedujemo imunitet nego još si dozvoljavamo da na otvorenoj sceni,i u 4će sutalji preksuta4tosbe dakle nekao samo što kao saborski zastupnici uživamo učinici da psjedmo imunitet nego još si dozvoljavamo dana otvrednoj ceni
2,/home/peterr/macocu/task8/transfer/Ow2zEc4S9Tk_14677.57-14697.13.wav,djeca u dobi od 8 godina i imaju šansu da budu posvojeni ali ova djeca iznad 8 godina nažalost ja to moram relativno grubo reći ali njihova šansa da budu posvojeni su je znatno znatno smanjena mi,djeca u dobi do8 godina i imaju šansu da bud posleni ali ova djeca iznad 8 godina na žalost ja to moram relativno grubo reći ali njihova šansa da bude da budu posvojjeni su je znatno znatno smanjena mi
3,/home/peterr/macocu/task8/transfer/41IJX-I7KUk_2061.17-2068.47.wav,koja će prekontrolirati što se je dogodilo 22. ožujka i da li su prekršeni zakoni rh,koja će prekontrolirati šta se dogodilo 20 drugog ožujka i dali su prekrišeni zakoni rh
4,/home/peterr/macocu/task8/transfer/Kt0nJgPw41E_2522.89-2542.02.wav,i donijeli odluku o istražnog povjerenstva govori da smo u okviru zakona spremni podržati osnivanjem istražnog povjerenstva koje bi se bavilo svim aspektima krize u agrokoru što je upravo bio slučaj kada smo tu odluku raspravili i onda je kasnije,i donijeli odluko snivanje stažnog povjerenstva govori da je smo u okviru zakona sprejemni podržati osnivanje istačnog povjerenstva koji bi se bavilo s fim aspektima krizeoagrokorušte upravo bio slučaj kada smo tu odluku raspravili i onda je kasnije
...,...,...,...
495,/home/peterr/macocu/task8/transfer/IRBTIpmhJe8_14598.0-14617.44.wav,da se pokušamo razmisliti da se vratimo na jedan upravni postupak i time smo još dalje ubrzali same procedure i zaključno reći ću još jedan prijedlog ukoliko budemo o tome raspravljali naime danas govorimo o važnosti investiranja i brzini ishođenja dokumenata za samo,da se pokušamo razmisliti da se vlatimu najdn uprani postupak i timi smo još dali je ubrzali same procedura i zaključno reće ću još jednu prijedlog u koliko budemo o tome raspravlji naime da nas govorimo o važnosti investiranja i brzini ishođenja dokumenata za samu
496,/home/peterr/macocu/task8/transfer/0n_W0iHVhpU_10409.72-10429.27.wav,i prema lokaciji nekretnine a ne prema mjestu prebivališta to je jako dobro i mislim da je to poticajno i za one jedinice lokalne samouprave na čijem prostoru je najviše upravo takvih nekretnina odnosno zgrada koje se iznajmljuju jer mislim da je to i opravdano i pravično premda je to izuzetak u odnosu na plaćanje poreza na dohodak koji se plaća prema mjestu,prema lokaciji nekretnine a neprea mjestu probivališta t je jako dobro ji mislim da je to potecajno i za one jedinicie lokalne samuprave na čm prostore najviše upravu takvih nekretnina koj se odosnog zgrada koji se iznamliui jer mislim da je to io pravdano i pravićno premda je to izruzetaku udnosu na plaćanje porezna dohoda koji se plaća pna mjestu
497,/home/peterr/macocu/task8/transfer/KnLfswocmwE_3789.82-3803.8.wav,naime nedopustivo je i neprihvatljivo je da se ova govornica koristi kako bi se preko leđa jednog čovjeka koji ima problema sa svojim poslodavcem a o kojem se problemu očitovalo dorh,naime nedopustivoje i neprihvatljivaje da vo govornica koristika koji se prekoleđa jedno